# Setup

In [1]:
sc.install_pypi_package("boto3")  # use boto3 to see book data in S3 bucket

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1654279618033_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3

s3 = boto3.resource('s3')
bucket = 'lzhang19'
bucket_resource = s3.Bucket(bucket)

for obj in bucket_resource.objects.all():
    if 'parquet' in obj.key and 'Books' in obj.key:
        print(obj.last_modified, "\t", round(obj.size * 1e-9), "GB\t", obj.key,
              "\n")
        
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Amazon Review Data

## Read from S3

In [3]:
data = spark.read.format('csv').options(header='true', inferSchema='true').load('s3://lzhang19/amazon_pet.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+----------+--------------------+-----+--------------+------------------+--------------------+--------------------+------------------+--------------------+
|_c0|index|product_id|               title|price|    num_rating|    overall_rating|       helpful_votes|                date|              star|                text|
+---+-----+----------+--------------------+-----+--------------+------------------+--------------------+--------------------+------------------+--------------------+
|  0|    0|B00006IX59|Chuckit! Classic ...| 8.72|46,941 ratings|4.7 out of 5 stars|385 people found ...|Reviewed in the U...|4.0 out of 5 stars|"After I broke my...|
|  1|    1|B00006IX59|Chuckit! Classic ...| 8.72|46,941 ratings|4.7 out of 5 stars|909 people found ...|Reviewed in the U...|3.0 out of 5 stars|Some (pointed) th...|
|  2|    2|B00006IX59|Chuckit! Classic ...| 8.72|46,941 ratings|4.7 out of 5 stars|100 people found ...|Reviewed in the U...|5.0 out of 5 stars|"I use this to th...|
|  3

## Sample

In [5]:
# scale up by commenting out the following line
# data = data.sample(fraction=0.001)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Clean Data

In [6]:
from pyspark.sql.functions import *

# clean overall rating
cleaned = data.withColumn('overall_rating', regexp_replace('overall_rating', ' out of 5 stars', '').cast('double'))
# clean star rating
cleaned = cleaned.withColumn('star', regexp_replace('star', ' out of 5 stars', '').cast('double'))
# clean number of ratings
cleaned = cleaned.withColumn('num_rating', regexp_replace('num_rating', ' ratings', ''))
cleaned = cleaned.withColumn('num_rating', regexp_replace('num_rating', ',', '').cast('integer'))
# clean helpful votes
cleaned = cleaned.withColumn('helpful_votes', regexp_replace('helpful_votes', 'One person found this helpful', '1'))
cleaned = cleaned.withColumn('helpful_votes', regexp_replace('helpful_votes', ' people found this helpful', ''))
cleaned = cleaned.withColumn('helpful_votes', col('helpful_votes').cast('integer'))
# seperate date and review location
cleaned = cleaned.withColumn("review_location", split(col("date"), " on ").getItem(0))
cleaned = cleaned.withColumn("review_location", regexp_replace('review_location', 'Reviewed in ', ''))
cleaned = cleaned.withColumn("review_date", split(col("date"), " on ").getItem(1))
cleaned = cleaned.withColumn("review_date", to_date(col("review_date"), 'MMMM d, yyyy').cast('string'))
# clean price
cleaned = cleaned.withColumn('price', col('price').cast('double'))
# drop un-useful columns
cleaned = cleaned.drop("_c0")
amazon_cleaned = cleaned.drop("date")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
amazon_cleaned.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+--------------------+-----+----------+--------------+-------------+----+--------------------+-----------------+-----------+
|index|product_id|               title|price|num_rating|overall_rating|helpful_votes|star|                text|  review_location|review_date|
+-----+----------+--------------------+-----+----------+--------------+-------------+----+--------------------+-----------------+-----------+
|    0|B00006IX59|Chuckit! Classic ...| 8.72|     46941|           4.7|          385| 4.0|"After I broke my...|the United States| 2019-05-18|
|    1|B00006IX59|Chuckit! Classic ...| 8.72|     46941|           4.7|          909| 3.0|Some (pointed) th...|the United States| 2015-07-15|
|    2|B00006IX59|Chuckit! Classic ...| 8.72|     46941|           4.7|          100| 5.0|"I use this to th...|the United States| 2019-06-04|
|    3|B00006IX59|Chuckit! Classic ...| 8.72|     46941|           4.7|          145|null|Combined with the...|the United States| 2017-05-19|
|    4

In [8]:
amazon_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- index: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: double (nullable = true)
 |-- num_rating: integer (nullable = true)
 |-- overall_rating: double (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- star: double (nullable = true)
 |-- text: string (nullable = true)
 |-- review_location: string (nullable = true)
 |-- review_date: string (nullable = true)

In [ ]:
location = (amazon_cleaned.groupBy('review_location').count().sort('review_location', ascending=False))
location.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
star = (amazon_cleaned.groupBy('star').count().sort('star', ascending=False))
star.show()

# COVID Data

## Read from S3

In [ ]:
covid_data = spark.read.format('csv').options(header='true', inferSchema='true').load('s3://lzhang19/us.csv')

## Clean Data

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

# increase cases per 
window = Window.partitionBy().orderBy("date")
cleaned = covid_data.withColumn("increase_cases", col("cases")-lag(col("cases"), 1, 0).over(window))
# increased deaths per day
cleaned = cleaned.withColumn("increase_deaths", col("deaths")-lag(col("deaths"), 1, 0).over(window))
# cast type
covid_cleaned = cleaned.withColumn("date", to_date(col("date"), 'yyyy-mm-dd').cast('string'))

In [ ]:
covid_cleaned.show(5)

In [ ]:
covid_cleaned.printSchema()